This code will create a STAC catalog by crawling over folders in the Cyverse Data Store and looking for geospatial assets (e.g., orthomosaics, point clouds, DEMs). 

Please launch 'JupyterLab_Geospatial' VICE app in Cyverse Discovery Environment
Open this Jupyter Notebook `STAC_creation_latest.ipynb`
Change the Kernel to `osgeo`
Use at least 16gb of RAM

In [1]:
#Install python libraries used in this script
%pip install rasterio==1.3.8 shapely==2.0.1 pystac==1.8.4 piexif==1.1.3 pandas==2.0.2 geopy==2.4.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
########Import libraries, modules, and classes into environment. ########
###########################

#Import the Path class from the pathlib module in Python. \
#The pathlib module is part of the Python Standard Library and provides \
#an object-oriented way to work with file system paths, \
#making it more convenient and readable than using traditional string-based file paths.

#The Path class is a high-level and flexible representation of file system paths. \
#It can be used for various purposes, such as constructing file system paths, navigating directories, \
#creating directories, reading or writing files, and more.
from pathlib import Path

#The os module is part of the Python Standard Library and provides a \
#collection of functions for interacting with the operating system. 
#This module allows you to perform various tasks such as creating, deleting, or modifying files and directories, \
#obtaining system information, and managing processes.
import os

#Import Optional typing hint
from typing import Optional

#ThreadPoolExecutor is a class from the 'concurrent.futures' module which comes from the Standard Python Library. 
#It provides a high level interface for asynchronously executing functions using threads. It can be used to execute \
#a certain number of functions in parallel, leveraging multiple threads. 
from concurrent.futures import ThreadPoolExecutor


#Import the pystac library into environment. Pystac (https://pystac.readthedocs.io/en/stable/index.html)
#is a library for reading and writing STAC stuff
import pystac 

#ProjectionExtension is a class within the module 'pystac.extensions.projection'. \
#We are using it to show the map projection of raster dataset
from pystac.extensions.projection import ProjectionExtension

#RasterExtension is a class within the module 'pystac.extensions.raster'
#we are using it to display various info about the raster
from pystac.extensions.raster import RasterExtension


from pystac.extensions.scientific import ScientificExtension
from pystac.extensions.scientific import Publication

from pystac.extensions.raster import RasterExtension


from pystac.provider import Provider
from pystac.provider import ProviderRole
import pystac.provider 
from pystac import Provider


#rasterio is a library to read and write raster data. It will be used in this script to extract information \
#from drone imagery geotiffs 
import rasterio
import rasterio.warp #used to convert coordinate of a raster bounding box

#Import the pdal lirbary into the environment. We use pdal (https://pdal.io/en/latest/) for reading LAS and LAZ files
#pdal is installed in the osgeo kernel, so we do not 'pip install pdal' in this script
import pdal

# Used to assist with pdal work
from osgeo import osr
from osgeo import ogr
from geopy.distance import geodesic


#Used to create a polygon of the raster bounds
from shapely.geometry import Polygon, mapping

#For creating a geojson output file of STAC items. This module comes in the Python Standard Library
import json

# datetime is a module in the Python's standard library. We need it to assign collection dates to each item or asset
from datetime import datetime

#We will use pandas to bring a csv file into a dataframe
import pandas

#We use the subprocess module when processing LAS/LAZ data. This prevents the kernel from \
#falsely getting flagged as unresponsive and automatically restarted (causing all work to be lost)
import subprocess

In [3]:
######This cell is for users to input manual metadata for imagery assets that will become STAC catalogs.#######
############################

#source_image_folder = '/data-store/iplant/home/jgillan/stac_test'
#source_image_folder = '/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019'
source_image_folder = '/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports'

# output folder (will overwrite existing files with matching output names)
#stac_output_directory = '/data-store/iplant/home/jgillan/STAC_drone/SRER_May2019'
#stac_output_directory = '/data-store/iplant/home/jgillan/stac_test/oct9'
stac_output_directory = '/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports/stac_test'

# Additional Metadata to add to items
platform = 'DJI Phantom 4 RTK'
license = 'CC-BY-SA-4.0'
items_mission_description = 'Open Forest Observatory Test Data'
pub_doi = '10.1002/ecs2.3649'
citation = 'Gillan, JK., GE Ponce-Campos, TL Swetnam, A Gorlier, P Heilman, MP McClaran. 2021. Innovations to expand drone data collection and analysis for rangeland monitoring. Ecosphere, 12(7)'

# collection definitions
collection_id = 'ofo_test' # needs to be folder-name compatible
collection_description = 'The imagery was part of the Ecostate Mapping project of 2019 at Santa Rita Experimental Range'

# top-level catalog definitions
catalog_id = 'Cyverse Remotely Sensed Imagery STAC Catalog'
catalog_description = 'This catalog includes all of the imagery assets the exist in Cyverse Data Store'


# create default datetime object for the collection - used when all items were collected on same date
default_datetime = datetime(year=2023, month=5, day=25, hour=12)

#If items were collected on different dates, then you should supply a csv that has the following columns:'Id', 'collection_date'
#df_collection_date = pandas.read_csv('/data-store/iplant/home/jgillan/stac_test/collection_date.csv')
#df_collection_date = pandas.read_csv('/data-store/iplant/home/shared/commons_repo/curated/Gillan_Ecosphere_2021/raster_products/May_2019/srer_may2019.csv')
#df_collection_date = pandas.read_csv('/data-store/iplant/home/shared/ofo/internal/david_data_for_Jeff/3_geometric-derived/metashape/reconstuction-000000/exports/collection_date.csv')
df_collection_date = pandas.read_csv('/data-store/iplant/home/jgillan/collection_date.csv')
                                                 
#Provider information
provider = Provider(name='Derek Young',
                       description="A provider that supplies example geospatial data.",
                       roles=[ProviderRole.PRODUCER, ProviderRole.PROCESSOR],
                       url='https://openforestobservatory.org/')

provider_dict = provider.to_dict()

#Email contact variable
contact_email = 'djyoung@ucdavis.edu'


# email host variables - DON'T CHANGE these unless you know what you're doing
smtp_host = '128.196.254.80'

In [4]:
#########Functions to extract the spatial resolution(ground sampline distance) from geotif imagery products \
##########These functions are using the 'rasterio' library.

# function to truncate spatial resolution to 3 decimal places
def trun_n_d(num,n):
    num_s = str(num)
    if 'e' in num_s or 'E' in num_s:
        return '{0:.{1}f}'.format(num,n)
    i,p,d = num_s.partition('.')
    return '.'.join([i,(d+'0'*n)[:n]])

# function to get the spatial resolution of a raster. It only works if the imagery products has a map projection, otherwise 
#it will return 0.00 
def spatial_resolution(raster):
    """extracts the XY Pixel Size"""
    t = raster.transform
    x = t[0]
    y = -t[4]
    x_trunc = trun_n_d(x, 3)
    y_trunc = trun_n_d(y, 3)
    return x_trunc, y_trunc

In [5]:
###########Function to calculate the bounding box and footprint of a geospatial raster dataset
######################

#This creates a function called 'get_bbox_and_footprint' for a raster we are calling 'dataset'
def get_bbox_and_footprint(dataset):

    # extract the bounding box of a raster using rasterio. '.bounds' is an attribute of 'rasterio.DatasetReader'
    #'bounds' returns the left, bottom, right, and top coordinates of a raster
    bounds = dataset.bounds
    
    #Transform the coordinate system of the raster bounds from it's orginial coordinate reference system \
    #to wgs84 which is also known as EPSG 4326. It uses the rasterio submodule 'rasterio.warp'
    bounds = rasterio.warp.transform_bounds(dataset.crs, 'EPSG:4326', 
                                            bounds.left, bounds.bottom, bounds.right, bounds.top)
   
    #The transformed bounds are then used to create a new rasterio.coords.BoundingBox object. \
    #The rasterio.coords.BoundingBox class is a convenient way to represent a bounding box \
    #with named attributes (left, bottom, right, and top) instead of using a tuple or a list. \
    #This makes the code more readable and easier to work with.
    bounds = rasterio.coords.BoundingBox(bounds[0], bounds[1], bounds[2], bounds[3])
    
    
    #The isinstance() function checks if the bounds variable is an instance of the \
    #rasterio.coords.BoundingBox class. If it is, it means that the bounds variable \
    #represents a bounding box with named attributes (left, bottom, right, and top).
    #If the bounds variable is an instance of rasterio.coords.BoundingBox, \
    #the code creates a list called bbox, which contains the bounding box coordinates \
    #in the following order: left, bottom, right, and top. This list is a more straightforward \
    #way to represent the bounding box as a sequence of coordinates.
    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        
    #If the bounds variable is not an instance of rasterio.coords.BoundingBox, \
    #the code assumes that it is a callable object (such as a function or a method) \
    #that returns the bounding box coordinates when called. In this case, the code \
    #calls the bounds() function and converts the returned bounding box coordinates \
    #to a list of floating-point numbers. This list is then assigned to the bbox variable.    
    else:
        bbox = [float(f) for f in bounds()]

    # create vertices and polygon from the bounding box coordinates. It uses the 'shapely.geometry' module \
    #from the shapely library. The output is the'footprint' variable which contains a shapely.geometry.Polygon object \
    #that represents the footprint of the raster dataset, based on its bounding box coordinates. \
    #This footprint can be used for tasks like spatial analysis, visualization, or overlaying with other geospatial data.
    footprint = Polygon([
        [bbox[0], bbox[1]],#left bottom
        [bbox[0], bbox[3]],#left top
        [bbox[2], bbox[3]],#right top
        [bbox[2], bbox[1]] #right bottom
    ])
    
    #Return the calculated bbox and the footprint as a dictionary using the mapping function \
    #(from the shapely.geometry module)
    return bbox, mapping(footprint)

In [6]:
##Function to get the spatial information on tiff imagery products

def tif_get_spatial_info(tif_file_path: Path) -> Optional[tuple]:

    # Open the individual file with rasterio
    ds = rasterio.open(tif_file_path)

    # Apply the function to get the bounding box (left, bottom, right, top) and make a footprint rectangle
    bbox, footprint = get_bbox_and_footprint(ds)

    # Extract the spatial resolution (gsd) of the image product using the function 'spatial_resolution'.
    x_res, y_res = spatial_resolution(ds)

    # Return the path, bounding box, the footprint, and the X and Y GSD (Ground Sample Distance)
    return tif_file_path, bbox, footprint, x_res, y_res, ds.shape, ds.crs.to_epsg(), pystac.MediaType.COG

In [7]:
#Determines the GSD (ground sampling distance) of an LAS/LAZ file

def las_spatial_resolution(bounding_box: tuple, num_points: int) -> tuple:
    
    min_y, min_x, max_y, max_x = bounding_box

    gsd_x = geodesic((min_x, min_y), (max_x, min_y)).meters / float(num_points)
    gsd_y = geodesic((min_x, min_y), (min_x, max_y)).meters / float(num_points)
    
    return gsd_x, gsd_y

In [8]:
####Returns the bounding box, footprint, and EPSG of the passed in LAS/LAZ file content

def get_las_bbox_footprint(las_srs_wkt: str, boundary_wkt: str, target_epsg: int=4326) -> list:
    # Returns a list containing the bounding box, and footprint geometries
    
    bounds_polygon = ogr.CreateGeometryFromWkt(boundary_wkt)

    # Translate the points if needed
    src_srs = osr.SpatialReference()
    src_srs.ImportFromWkt(las_srs_wkt)

    geom_epsg = int(src_srs.GetAttrValue('AUTHORITY', 1))
    if geom_epsg != target_epsg:
        # Set up the transformation
        dst_srs = osr.SpatialReference()
        dst_srs.ImportFromEPSG(target_epsg)

        transform = osr.CoordinateTransformation(src_srs, dst_srs)
        
        # Perform the transformation
        transl_polygon = bounds_polygon.Clone()
        transl_polygon.Transform(transform)
        
        bounds_polygon = transl_polygon

    # Get the envelope of the polygon and re-arrange to expected order
    return_bbox = bounds_polygon.GetEnvelope()
    return_bbox = (return_bbox[2], return_bbox[0], return_bbox[3], return_bbox[1])

    # Create the polygon of the footprint
    footprint = Polygon([
        [return_bbox[0], return_bbox[1]],#left bottom
        [return_bbox[0], return_bbox[3]],#left top
        [return_bbox[2], return_bbox[3]],#right top
        [return_bbox[2], return_bbox[1]] #right bottom
    ])

    return return_bbox, footprint, geom_epsg


In [9]:
####Function to get the spatial information on the point cloud files

#Return the information on the LAS/LAZ files
def las_get_spatial_info(las_file_path: Path) -> Optional[tuple]:

    # Folder to write temporary files to
    root_dir = '/tmp'
        
    # Configure the processing pipeline for the data we will want
    pipeline_json  = """
    {
        "pipeline": [
            "%s",
            {
                "type" : "filters.hexbin"
            }
        ]
    }
    """ % las_file_path

    # Get the filenames for the pipeline text, and the output of the pipeline
    base_filename, file_ext = os.path.splitext(os.path.basename(las_file_path))
    pipeline_path = os.path.join(root_dir, base_filename + '_pipeline.json')
    out_path = os.path.join(root_dir, base_filename + '.json')
    with open(pipeline_path, "w") as outfile:
        outfile.write(pipeline_json)

    # Make the call to run the pdal app to process the pipeline
    result = subprocess.run(['/opt/conda/envs/osgeo/bin/pdal', 'pipeline', pipeline_path, '--metadata', out_path], stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)

    # Load the output of the pipeline
    with open(out_path, 'r') as infile:
        metadata = json.load(infile)

    # Clean up the temporary files
    os.remove(pipeline_path)
    os.remove(out_path)

    # Assign to variable to make it easier for a developer
    readers = metadata['stages']['readers.las']
    hexbin = metadata['stages']['filters.hexbin']

    # Get the bounding box in the correct coordinate system
    bbox, footprint, epsg = get_las_bbox_footprint(readers['srs']['wkt'], hexbin['boundary'])

    # Extract the spatial resolution (gsd) of the image product using the function 'las_spatial_resolution'.
    x_res, y_res = las_spatial_resolution(bbox, readers['count'])

    # Return the path, bounding box, the footprint, and the X and Y GSD (Ground Sample Distance)
    return las_file_path, bbox, mapping(footprint), x_res, y_res, (None, None), epsg, 'application/vnd.laszip' if file_ext == '.laz' else 'application/vnd.las'

In [10]:

###This starts the for loop to crawl through a folder, find imagery assets, and generate STAC json metadata files for each item

os.environ['PROJ_DATA'] = '/opt/conda/envs/osgeo/share/proj'

# Disable a warning message
ogr.UseExceptions()

# Get a list of geospatial files (in this case located in Cyverse data store)
folder = Path(source_image_folder)


# Create an extension mapping dictionary to assist with the file discovery and multithreading
# Each extension maps to the function to call
ext_map = {'.tif': tif_get_spatial_info,
           '.las': las_get_spatial_info,
           '.laz': las_get_spatial_info
          }

# Load the files of interest
files = list()
for one_filter in ext_map.keys():
    files = files + list(folder.rglob('*' + one_filter))

# Initialize variables to hold the results
items_dict = {}
all_items = []

# Create a ThreadPoolExecutor with 2 worker threads to load all the spatial data
with ThreadPoolExecutor(max_workers=2) as executor:
    # Submit each file to the executor using the file extension to determine which function to call
    future_results = [executor.submit(ext_map[os.path.splitext(file)[1]], file) for file in files]

    # Get the results as they become available
    all_results = [future.result() for future in future_results]

    
# Loop through each item in the folder and do several things
for result in all_results:

    #Get the specific results from processing one file
    file, bbox, footprint, x_res, y_res, width_height, srid, media_type = result

    # the ID (name) for each indivual file
    idx = file.stem
    
    
    
    ##The following 3 lines are for assigning the date of imagery collection to each of the item by matching IDs from a csv file
    
    
    # Within the for loop that we are in, this line looks at the 'Id' column of the csv file (imported into python as a pandas DataFrame).
    # It takes the Id name of the current geotiff file and looks for a match within the Id column within 'df_collection_date'. 
    # If it finds a matching ID, it returns the info for the entire row. IDs in the dataframe are stored as strings. 
    
    collection_time = df_collection_date[df_collection_date.Id == str(idx)]
    
    # From the matched row, this command will return the value within the 'collection_date' column as a 2D numpy array 
    dates = collection_time.collection_date.values
    
    # Get the plot name of the item
    plot = collection_time['plot'].iloc[0]
    
    # Convert the date into a 'datetime' object (e.g., 2019-05-19)
    datess = datetime.strptime(dates[0], '%Y-%m-%d')

    # Check if an item with this plot name already exists
    if plot in items_dict:
        #if it does, get that item
        item = items_dict[plot]
        
    # If this is an item with a new plot name 
    else:    
        # create a STAC item for each individual file 
        item = pystac.Item(id=plot,
                geometry=footprint,
                bbox=bbox,
                datetime=datess,
                #datetime=default_datetime,
                stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
                                 'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
                 
                properties={'gsd': x_res,
                        'platform': platform,
                        'license': license,
                        'mission': items_mission_description,
                        'sci:doi': pub_doi,
                        'sci:citation': citation,
                        'providers': [provider_dict]}
        )
    items_dict[plot] = item
    
    # Adding the map projection extension to each item. Otherwise, the projection info will not display 
    params = {
        'bbox': bbox,
        'geometry': footprint
        }
    if None not in width_height:
        params['shape'] = width_height  #ds.shape,
    ProjectionExtension.ext(item).apply(srid, #ds.crs.to_epsg(),
                                        **params)
                                        #transform = [float(getattr(ds.transform, letter)) for letter in 'abcdef']
                                        #)
   
    # Add the asset link to the item and define the type of geospatial format it is
    item.add_asset(
        key=idx,
        asset=pystac.Asset(
            href=file.as_posix(), 
            media_type=media_type, #pystac.MediaType.COG
            roles='data'
            #extra_fields=asset_ext
        )
    )

    # Add each STAC item to a list of all the items
    all_items.append(item)
    all_items = list(items_dict.values())

print('Done processing files')

Done processing files


In [11]:
#######Create and describe a STAC Collection 
##########################

# the geographic extent of all the items added
item_extents = pystac.Extent.from_items(all_items)

# creating the collection
collection = pystac.Collection(id=collection_id,
                               description=collection_description,
                               extent=item_extents,
                               license=license)

# add all STAC item to the STAC collection
for item in all_items:
    collection.add_item(item)

In [12]:
# print the number of STAC items that were added to the STAC catalog    
print(len(list(collection.get_items())))

# describe the items in the STAC catalog
collection.describe()

1
* <Collection id=ofo_test>
  * <Item id=15_g2>


In [13]:
##########Save the collection and its child items to output directory (ABSOLUTE LINKS FOR STAC API)

collection.normalize_hrefs(stac_output_directory)

collection.save(catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED)


In [14]:
#########Combine all of the STAC item metadata into a single geojson and output to directory
####################

# Create an empty GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": []
}

# Iterate through all the items in the collection and convert them to GeoJSON Features
for item in collection.get_all_items():
    # Get the STAC Item as a dictionary
    item_dict = item.to_dict()

    # Convert the STAC Item to a GeoJSON Feature
    feature = {
        "type": "Feature",
        "collection": item_dict["collection"],
        "stac_version": item_dict["stac_version"],
        "stac_extensions": item_dict["stac_extensions"],
        "id": item_dict["id"],
        "geometry": item_dict["geometry"],
        "bbox": item_dict["bbox"],
        "properties": item_dict["properties"],
        "assets": item_dict["assets"]
    }

    # Add the GeoJSON Feature to the FeatureCollection
    geojson["features"].append(feature)


# Make sure the output_directory ends with a path separator
if not stac_output_directory.endswith("/"):
    stac_output_directory += "/"

output_file_path = stac_output_directory + "index.geojson"

# Write the GeoJSON FeatureCollection to a file
with open(output_file_path, "w") as f:
    json.dump(geojson, f, indent=4)        

    

# Define the string to find and the string to replace it with
string_to_find = "/data-store"
string_to_replace = "https://data.cyverse.org/dav-anon"

# Read the file into a string
with open(output_file_path, "r") as f:
    file_content = f.read()

# Perform the find-and-replace operation
file_content = file_content.replace(string_to_find, string_to_replace)

# Write the result back to the file
with open(output_file_path, "w") as f:
    f.write(file_content)
    
    
   
    

To make the saved catalog available, you will need to:
-  share the output folder to \"public\" using the [Discovery Environment](https://de.cyverse.org/) Data tab
- add your catalog to the CyVerse [master catalog](/iplant/home/jgillan/stac.cyverse.org/cyverse_stac_catalog/catalog.json)


In [15]:
#%pip install python-irodsclient

The following will share the folder containing your catalog with the CyVerse `public` user using read-only permissions

You will be prompted to enter your CyVerse username and password. Be sure to use the *Enter* key for each prompt

In [16]:
#import os
#from getpass import getpass
#from irods.access import iRODSAccess
#from irods.session import iRODSSession

#irods_username = input('Enter your CyVerse user name:')
#irods_password = getpass('Enter your CyVerse password:')

#sess = iRODSSession(host='data.cyverse.org', port=1247, user=irods_username, password=irods_password, zone='iplant')

#user = sess.users.get(sess.username, sess.zone)

# add the needed users for linking
#acl_path = '/' + os.path.join(*(stac_output_directory.split(os.path.sep)[2:]))
#acl = iRODSAccess('read', acl_path, 'anonymous', user.zone)
#sess.acls.set(acl, recursive=True)

#sess.cleanup()

#print("Folder updated")

**Almost Done!**

To have a *newly created* catalog added to the global CyVerse catalog, update `your_email` using your email address
and send an email using the following:

In [17]:
#your_email = '<someone>@arizona.edu'

# Import smtplib for the actual sending function
#import smtplib

# Import the email modules we'll need
#from email.mime.text import MIMEText

#msg = MIMEText("Please add the following STAC catalog to the main CyVerse catalog: " + 
               #"https://data.cyverse.org/dav-anon/" + os.path.join(*(stac_output_directory.split(os.path.sep)[2:])) +
               #"/catalog.json")
#msg['Subject'] = 'Add new STAC catalog to main CyVerse catalog'
#msg['From'] = your_email
#msg['To'] = contact_email

# Send the message via our own SMTP server, but don't include the
# envelope header.
#smtp = smtplib.SMTP(smtp_host)
#smtp.sendmail(your_email, [contact_email], msg.as_string())
#smtp.quit()
